In [1]:
import numpy as np
import pandas as pd
#import datetime as dt
import pymongo
import json

In [7]:
#import os
from bs4 import BeautifulSoup as bs
#import requests
from splinter import Browser
import time

In [3]:
# Url to get all boxscores for 2017-2018 regular season games
nba_17_18_boxscores_url = "https://stats.nba.com/teams/boxscores-advanced/?Season=2017-18&SeasonType=Regular%20Season"

In [5]:
# create splinter browser
def init_browser():
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
    return browser

browser = init_browser()

In [6]:
# visit website
browser.visit(nba_17_18_boxscores_url)
time.sleep(2)

In [8]:
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")

In [9]:
# select to display all games
sel_all_path.click()

In [10]:
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')

In [11]:
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))

In [12]:
# drop season column since unneeded
new_df = df[0].drop(columns=['Season'])

In [13]:
# export to json
items = new_df.to_json(orient='records')

In [14]:
# create mongo db connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [15]:
# connect to my nba data db
db = client.nba_data_db

In [16]:
# load json string to json
items_db = json.loads(items)

In [17]:
# drop any items in training data collection
db.training_data.drop()

In [18]:
# insert data to collection
db.training_data.insert_many(items_db)